In [134]:
import pandas as pd
from statsmodels.tsa.ar_model import AutoReg
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

In [135]:
df_5 = pd.read_csv('./in/copper_returns_5d_final.csv')
df_10 = pd.read_csv('./in/copper_returns_10d_final.csv')
df_20 = pd.read_csv('./in/copper_returns_20d_final.csv')

df_5
# print(df_10)
# print(df_20)

,Date,LMCADS03,LMCADY,DXY,SPX,BCOM,MXWD,XAU,XAG,LMCADY_acu_5d,LMCADY_std_5d
0,2024-03-25,0.001241,0.001047,-0.000767,0.002808,0.006127,0.001182,-0.003199,0.009327,-0.001010,0.001367
1,2024-03-22,-0.000733,-0.000182,0.002015,0.003064,-0.005829,0.002399,-0.002942,-0.000409,-0.001191,0.001363
2,2024-03-21,0.009474,0.009322,-0.004022,0.001404,0.007491,0.002585,0.007338,0.002853,0.008130,0.004511
3,2024-03-20,-0.002514,-0.002130,-0.005961,-0.003226,-0.001953,-0.006740,0.002320,0.033895,0.008524,0.004423
4,2024-03-19,0.005432,0.005893,0.004159,-0.008826,0.003441,-0.006092,-0.013172,-0.026096,0.013951,0.004700
...,...,...,...,...,...,...,...,...,...,...,...
5534,2003-01-07,-0.009774,-0.009724,0.006872,0.014287,-0.004085,0.014765,-0.018041,-0.010309,-0.026799,0.009751
5535,2003-01-06,0.004627,0.004489,-0.005850,0.006588,0.002701,0.008892,0.011519,0.020312,-0.011388,0.009986
5536,2003-01-03,-0.010746,-0.010210,0.004903,-0.021980,0.016248,-0.018378,0.000000,-0.001531,-0.015156,0.010507
5537,2003-01-02,-0.013035,-0.013388,0.004977,0.000484,-0.014186,-0.003775,-0.014804,-0.014724,-0.040276,0.007154


In [136]:
df_5['Date'] = pd.to_datetime(df_5['Date'])
df_5.set_index('Date', inplace=True)

In [137]:
df_5['LMCADY_acu_5d'] = df_5['LMCADY_acu_5d'].shift(-1)
df_5['LMCADY_std_5d'] = df_5['LMCADY_std_5d'].shift(-1)

df_5.dropna(inplace=True)
df_5

,LMCADS03,LMCADY,DXY,SPX,BCOM,MXWD,XAU,XAG,LMCADY_acu_5d,LMCADY_std_5d
Date,,,,,,,,,,
2024-03-25,0.001241,0.001047,-0.000767,0.002808,0.006127,0.001182,-0.003199,0.009327,-0.001191,0.001363
2024-03-22,-0.000733,-0.000182,0.002015,0.003064,-0.005829,0.002399,-0.002942,-0.000409,0.008130,0.004511
2024-03-21,0.009474,0.009322,-0.004022,0.001404,0.007491,0.002585,0.007338,0.002853,0.008524,0.004423
2024-03-20,-0.002514,-0.002130,-0.005961,-0.003226,-0.001953,-0.006740,0.002320,0.033895,0.013951,0.004700
2024-03-19,0.005432,0.005893,0.004159,-0.008826,0.003441,-0.006092,-0.013172,-0.026096,0.025901,0.006343
...,...,...,...,...,...,...,...,...,...,...
2003-01-08,-0.010876,-0.011443,-0.000687,-0.019028,-0.013837,-0.014799,0.002495,0.002066,-0.026799,0.009751
2003-01-07,-0.009774,-0.009724,0.006872,0.014287,-0.004085,0.014765,-0.018041,-0.010309,-0.011388,0.009986
2003-01-06,0.004627,0.004489,-0.005850,0.006588,0.002701,0.008892,0.011519,0.020312,-0.015156,0.010507


In [138]:
target = 'LMCADY_acu_5d'
train_data, test_data = train_test_split(df_5[target], test_size=0.15, shuffle=False)

In [139]:
model_ar = AutoReg(train_data, lags=1, old_names=False)
model_ar_fitted = model_ar.fit()

c:\Users\gapuj\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\gapuj\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [140]:
predictions = model_ar_fitted.predict(start=len(train_data), end=len(train_data) + len(test_data) - 1, dynamic=False)

c:\Users\gapuj\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\gapuj\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
c:\Users\gapuj\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\tsa\deterministic.py:307: UserWarning: Only PeriodIndexes, DatetimeIndexes with a frequency set, RangesIndexes, and Index with a unit increment support extending. The index is set will contain the position relative to the data length.
  fcast_index = self._extend_index(index, steps, forecast_index)


In [141]:
mse = mean_squared_error(test_data, predictions)
rmse = np.sqrt(mse)

In [142]:
print(model_ar_fitted.summary()) 
print("RMSE: {:.5f}".format(rmse))

                            AutoReg Model Results                             
Dep. Variable:          LMCADY_acu_5d   No. Observations:                 4707
Model:                     AutoReg(1)   Log Likelihood               11230.071
Method:               Conditional MLE   S.D. of innovations              0.022
Date:                Fri, 19 Apr 2024   AIC                         -22454.143
Time:                        20:30:48   BIC                         -22434.773
Sample:                             1   HQIC                        -22447.333
                                 4707                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const             1.744e-05      0.000      0.054      0.957      -0.001       0.001
LMCADY_acu_5d.L1     0.7787      0.009     85.133      0.000       0.761       0.797
                            